This is the notebook for training and evaluating an ETM model.
Before running this notebook, run
1. 1.0-rw-preProcessText_makeBOW
2. 1.0-rw-preProcessText2_embeddings

There are some issues with this code.
First, the nearest neighbor results are very poor.
Second, there seems to be an error in the definition of the evaluate function below, because it uses the test data for validation AND for testing.  See the function for comments.

Pieces still missing:
Once the model is built, still need to extract theta.  Code for this is still forthcoming.
The original (based on Adji B. Dieng's code) text preprocessing in the preprocessText_makeBOW 
has been modified in order
to keep track of the indices of train/val/test documents, so that the topics, once
extracted from the model theta, can be correctly assigned to the original document.

In [1]:
# from Adji Dieng GitHub ETM

from __future__ import print_function

import argparse
import torch
import pickle 
import numpy as np 
import os 
import math 
import random 
import sys
import matplotlib.pyplot as plt  
import data
import scipy.io

from torch import nn, optim
from torch.nn import functional as F

from etm import ETM
from utils import nearest_neighbors, get_topic_coherence, get_topic_diversity


In [2]:
parser = argparse.ArgumentParser(description='The Embedded Topic Model')



In [3]:
### data and file related arguments
parser.add_argument('--dataset', type=str, default='min_df_3', help='name of corpus') # default='20ng'
parser.add_argument('--data_path', type=str, default='data/min_df_3', help='directory containing data') #default='data/20ng'
parser.add_argument('--emb_path', type=str, default='data/min_df_3_embeddings.txt', help='directory containing word embeddings')
parser.add_argument('--save_path', type=str, default='./results', help='path to save results')
parser.add_argument('--batch_size', type=int, default=1000, help='input batch size for training')


### model-related arguments
parser.add_argument('--num_topics', type=int, default=150, help='number of topics')
parser.add_argument('--rho_size', type=int, default=300, help='dimension of rho')
parser.add_argument('--emb_size', type=int, default=300, help='dimension of embeddings')
parser.add_argument('--t_hidden_size', type=int, default=800, help='dimension of hidden space of q(theta)')
parser.add_argument('--theta_act', type=str, default='relu', help='tanh, softplus, relu, rrelu, leakyrelu, elu, selu, glu)')
parser.add_argument('--train_embeddings', type=int, default=0, help='whether to fix rho or train it') # default = 0

### optimization-related arguments
parser.add_argument('--lr', type=float, default=0.005, help='learning rate')
parser.add_argument('--lr_factor', type=float, default=4.0, help='divide learning rate by this...')
parser.add_argument('--epochs', type=int, default=100, help='number of epochs to train...150 for 20ng 100 for others')
parser.add_argument('--mode', type=str, default='train', help='train or eval model') # default='train'
parser.add_argument('--optimizer', type=str, default='adam', help='choice of optimizer')
parser.add_argument('--seed', type=int, default=2019, help='random seed (default: 1)')
parser.add_argument('--enc_drop', type=float, default=0.0, help='dropout rate on encoder')
parser.add_argument('--clip', type=float, default=0.0, help='gradient clipping')
parser.add_argument('--nonmono', type=int, default=10, help='number of bad hits allowed')
parser.add_argument('--wdecay', type=float, default=1.2e-6, help='some l2 regularization')
parser.add_argument('--anneal_lr', type=int, default=0, help='whether to anneal the learning rate or not')
parser.add_argument('--bow_norm', type=int, default=1, help='normalize the bows or not')

### evaluation, visualization, and logging-related arguments
parser.add_argument('--num_words', type=int, default=10, help='number of words for topic viz')
parser.add_argument('--log_interval', type=int, default=2, help='when to log training')
parser.add_argument('--visualize_every', type=int, default=10, help='when to visualize results')
parser.add_argument('--eval_batch_size', type=int, default=1000, help='input batch size for evaluation')
parser.add_argument('--load_from', type=str, default='', help='the name of the ckpt to eval from')
parser.add_argument('--tc', type=int, default=0, help='whether to compute topic coherence or not')
parser.add_argument('--td', type=int, default=0, help='whether to compute topic diversity or not')

_StoreAction(option_strings=['--td'], dest='td', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help='whether to compute topic diversity or not', metavar=None)

In [4]:
args, unknown = parser.parse_known_args()

#args = parser.parse_args()
# based on solution from:
# https://stackoverflow.com/questions/48796169/how-to-fix-ipykernel-launcher-py-error-unrecognized-arguments-in-jupyter

print(args)

Namespace(anneal_lr=0, batch_size=1000, bow_norm=1, clip=0.0, data_path='data/min_df_3', dataset='min_df_3', emb_path='data/min_df_3_embeddings.txt', emb_size=300, enc_drop=0.0, epochs=100, eval_batch_size=1000, load_from='', log_interval=2, lr=0.005, lr_factor=4.0, mode='train', nonmono=10, num_topics=150, num_words=10, optimizer='adam', rho_size=300, save_path='./results', seed=2019, t_hidden_size=800, tc=0, td=0, theta_act='relu', train_embeddings=0, visualize_every=10, wdecay=1.2e-06)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('\n')
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    print("yay")
    torch.cuda.manual_seed(args.seed)
else:
    print("nope")



nope


In [6]:
## get data
# 1. vocabulary
vocab, train, valid, test = data.get_data(os.path.join(args.data_path))
vocab_size = len(vocab)
args.vocab_size = vocab_size
print(args.vocab_size)

10616


In [7]:
# 1. training data
train_tokens = train['tokens']
train_counts = train['counts']
args.num_docs_train = len(train_tokens)
print(args.num_docs_train)

1657


In [8]:
# 2. dev set
valid_tokens = valid['tokens']
valid_counts = valid['counts']
args.num_docs_valid = len(valid_tokens)
print(args.num_docs_valid)

# 3. test data
test_tokens = test['tokens']
test_counts = test['counts']
args.num_docs_test = len(test_tokens)
test_1_tokens = test['tokens_1']
test_1_counts = test['counts_1']
args.num_docs_test_1 = len(test_1_tokens)
test_2_tokens = test['tokens_2']
test_2_counts = test['counts_2']
args.num_docs_test_2 = len(test_2_tokens)
print("test docs")
print(args.num_docs_test)
print(args.num_docs_test_1)
print(args.num_docs_test_2)


99
test docs
196
196
196


In [9]:
embeddings = None
if not args.train_embeddings:
    emb_path = args.emb_path
#    vect_path = os.path.join(args.data_path.split('/')[0], 'embeddings.pkl')   
    vectors = {}
    with open(emb_path, 'rb') as f:
        for l in f:
            line = l.decode().split()
            word = line[0]
            if word in vocab:
                vect = np.array(line[1:]).astype(np.float)
                vectors[word] = vect
    embeddings = np.zeros((vocab_size, args.emb_size))
    words_found = 0
    for i, word in enumerate(vocab):
        try: 
            embeddings[i] = vectors[word]
            words_found += 1
        except KeyError:
            embeddings[i] = np.random.normal(scale=0.6, size=(args.emb_size, ))
    embeddings = torch.from_numpy(embeddings).to(device)
    args.embeddings_dim = embeddings.size()

print('=*'*100)
print('Training an Embedded Topic Model on {} with the following settings: {}'.format(args.dataset.upper(), args))
print('=*'*100)


=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
Training an Embedded Topic Model on MIN_DF_3 with the following settings: Namespace(anneal_lr=0, batch_size=1000, bow_norm=1, clip=0.0, data_path='data/min_df_3', dataset='min_df_3', emb_path='data/min_df_3_embeddings.txt', emb_size=300, embeddings_dim=torch.Size([10616, 300]), enc_drop=0.0, epochs=100, eval_batch_size=1000, load_from='', log_interval=2, lr=0.005, lr_factor=4.0, mode='train', nonmono=10, num_docs_test=196, num_docs_test_1=196, num_docs_test_2=196, num_docs_train=1657, num_docs_valid=99, num_topics=150, num_words=10, optimizer='adam', rho_size=300, save_path='./results', seed=2019, t_hidden_size=800, tc=0, td=0, theta_act='relu', train_embeddings=0, visualize_every=10, vocab_size=10616, wdecay=1.2e-06)
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=

In [10]:
## define checkpoint
if not os.path.exists(args.save_path):
    os.makedirs(args.save_path)

if args.mode == 'eval model':
    ckpt = args.load_from
else:
    ckpt = os.path.join(args.save_path, 
        'etm_{}_K_{}_Htheta_{}_Optim_{}_Clip_{}_ThetaAct_{}_Lr_{}_Bsz_{}_RhoSize_{}_trainEmbeddings_{}'.format(
        args.dataset, args.num_topics, args.t_hidden_size, args.optimizer, args.clip, args.theta_act, 
            args.lr, args.batch_size, args.rho_size, args.train_embeddings))


In [11]:
## define model and optimizer
model = ETM(args.num_topics, vocab_size, args.t_hidden_size, args.rho_size, args.emb_size, 
                args.theta_act, embeddings, args.train_embeddings, args.enc_drop).to(device)

print('model: {}'.format(model))

model: ETM(
  (t_drop): Dropout(p=0.0)
  (theta_act): ReLU()
  (alphas): Linear(in_features=300, out_features=150, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=10616, out_features=800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=800, out_features=800, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=800, out_features=150, bias=True)
  (logsigma_q_theta): Linear(in_features=800, out_features=150, bias=True)
)


In [12]:
if args.optimizer == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.wdecay)
elif args.optimizer == 'adagrad':
    optimizer = optim.Adagrad(model.parameters(), lr=args.lr, weight_decay=args.wdecay)
elif args.optimizer == 'adadelta':
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr, weight_decay=args.wdecay)
elif args.optimizer == 'rmsprop':
    optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.wdecay)
elif args.optimizer == 'asgd':
    optimizer = optim.ASGD(model.parameters(), lr=args.lr, t0=0, lambd=0., weight_decay=args.wdecay)
else:
    print('Defaulting to vanilla SGD')
    optimizer = optim.SGD(model.parameters(), lr=args.lr)


In [13]:
def train(epoch):
    model.train()
    acc_loss = 0
    acc_kl_theta_loss = 0
    cnt = 0
    indices = torch.randperm(args.num_docs_train)
    indices = torch.split(indices, args.batch_size)
    for idx, ind in enumerate(indices):
        optimizer.zero_grad()
        model.zero_grad()
        data_batch = data.get_batch(train_tokens, train_counts, ind, args.vocab_size, device)
        sums = data_batch.sum(1).unsqueeze(1)
        if args.bow_norm:
            normalized_data_batch = data_batch / sums
        else:
            normalized_data_batch = data_batch
        recon_loss, kld_theta = model(data_batch, normalized_data_batch)
        total_loss = recon_loss + kld_theta
        total_loss.backward()

        if args.clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()

        acc_loss += torch.sum(recon_loss).item()
        acc_kl_theta_loss += torch.sum(kld_theta).item()
        cnt += 1

        if idx % args.log_interval == 0 and idx > 0:
            cur_loss = round(acc_loss / cnt, 2) 
            cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
            cur_real_loss = round(cur_loss + cur_kl_theta, 2)

            print('Epoch: {} .. batch: {}/{} .. LR: {} .. KL_theta: {} .. Rec_loss: {} .. NELBO: {}'.format(
                epoch, idx, len(indices), optimizer.param_groups[0]['lr'], cur_kl_theta, cur_loss, cur_real_loss))
    
    cur_loss = round(acc_loss / cnt, 2) 
    cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
    cur_real_loss = round(cur_loss + cur_kl_theta, 2)
    print('*'*100)
    print('Epoch----->{} .. LR: {} .. KL_theta: {} .. Rec_loss: {} .. NELBO: {}'.format(
            epoch, optimizer.param_groups[0]['lr'], cur_kl_theta, cur_loss, cur_real_loss))
    print('*'*100)


In [61]:
def visualize(m, show_emb=True):
    if not os.path.exists('./results'):
        os.makedirs('./results')

    m.eval()

    queries = ['covid', 'refugee', 'resource', 'ventilator', 'women',
                'men', 'money', 'society', 'health', 'people', 'family']
    

    ## visualize topics using monte carlo
    with torch.no_grad():
        print('#'*100)
        print('Visualize topics...')
        topics_words = []
        gammas = m.get_beta()
        for k in range(args.num_topics):
            gamma = gammas[k]
            top_words = list(gamma.cpu().numpy().argsort()[-args.num_words+1:][::-1])
            topic_words = [vocab[a] for a in top_words]
            topics_words.append(' '.join(topic_words))
            print('Topic {}: {}'.format(k, topic_words))

        if show_emb:
            ## visualize word embeddings by using V to get nearest neighbors
            print('#'*100)
            print('Visualize word embeddings by using output embedding matrix')
            try:
                embeddings = m.rho.weight  # Vocab_size x E
            except:
                embeddings = m.rho         # Vocab_size x E
            neighbors = []
            for word in queries:
                print('word: {} .. neighbors: {}'.format(
                    word, nearest_neighbors(word, embeddings, vocab)))
            print('#'*100)


In [62]:
def evaluate(m, source, tc=False, td=False):
    """Compute perplexity on document completion.
    """
    m.eval()
    with torch.no_grad():
        if source == 'val':   # this if else statement defines tokens and counts, but they are not used anywhere
            indices = torch.split(torch.tensor(range(args.num_docs_valid)), args.eval_batch_size)
            tokens = valid_tokens
            counts = valid_counts
        else: 
            indices = torch.split(torch.tensor(range(args.num_docs_test)), args.eval_batch_size)
            tokens = test_tokens
            counts = test_counts

        ## get \beta here
        beta = m.get_beta()

        ### do dc and tc here
        acc_loss = 0
        cnt = 0
        indices_1 = torch.split(torch.tensor(range(args.num_docs_test_1)), args.eval_batch_size) # it looks like the test set is _always_ being used
        for idx, ind in enumerate(indices_1):
            ## get theta from first half of docs
            data_batch_1 = data.get_batch(test_1_tokens, test_1_counts, ind, args.vocab_size, device)
            sums_1 = data_batch_1.sum(1).unsqueeze(1)
            if args.bow_norm:
                normalized_data_batch_1 = data_batch_1 / sums_1
            else:
                normalized_data_batch_1 = data_batch_1
            theta, _ = m.get_theta(normalized_data_batch_1)

            ## get prediction loss using second half
            data_batch_2 = data.get_batch(test_2_tokens, test_2_counts, ind, args.vocab_size, device)
            sums_2 = data_batch_2.sum(1).unsqueeze(1)
            res = torch.mm(theta, beta)
            preds = torch.log(res)
            recon_loss = -(preds * data_batch_2).sum(1)
            
            loss = recon_loss / sums_2.squeeze()
            loss = loss.mean().item()
            acc_loss += loss
            cnt += 1
        cur_loss = acc_loss / cnt
        ppl_dc = round(math.exp(cur_loss), 1)
        print('*'*100)
        print('{} Doc Completion PPL: {}'.format(source.upper(), ppl_dc))
        print('*'*100)
        if tc or td:
            beta = beta.data.cpu().numpy()
            if tc:
                print('Computing topic coherence...')
                get_topic_coherence(beta, train_tokens, vocab)
            if td:
                print('Computing topic diversity...')
                get_topic_diversity(beta, 25)
        return ppl_dc


In [63]:
if args.mode == 'train': # 'eval model': #
    ## train model on data 
    best_epoch = 0
    best_val_ppl = 1e9
    all_val_ppls = []
    print('\n')
    print('Visualizing model quality before training...')
    visualize(model)
    print('\n')
    for epoch in range(1, args.epochs):
        train(epoch)
        val_ppl = evaluate(model, 'val')
        if val_ppl < best_val_ppl:
            with open(ckpt, 'wb') as f:
                torch.save(model, f)
            best_epoch = epoch
            best_val_ppl = val_ppl
        else:
            ## check whether to anneal lr
            lr = optimizer.param_groups[0]['lr']
            if args.anneal_lr and (len(all_val_ppls) > args.nonmono and val_ppl > min(all_val_ppls[:-args.nonmono]) and lr > 1e-5):
                optimizer.param_groups[0]['lr'] /= args.lr_factor
        if epoch % args.visualize_every == 0:
            visualize(model)
        all_val_ppls.append(val_ppl)
    with open(ckpt, 'rb') as f:
        model = torch.load(f)
    model = model.to(device)
    val_ppl = evaluate(model, 'val')
else:   
    with open(ckpt, 'rb') as f:
        model = torch.load(f)
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        ## get document completion perplexities
        test_ppl = evaluate(model, 'test', tc = 1, td = 1) # tc=args.tc, td=args.td) #

        ## get most used topics
        indices = torch.tensor(range(args.num_docs_train))
        indices = torch.split(indices, args.batch_size)
        thetaAvg = torch.zeros(1, args.num_topics).to(device)
        thetaWeightedAvg = torch.zeros(1, args.num_topics).to(device)
        cnt = 0
        for idx, ind in enumerate(indices):
            data_batch = data.get_batch(train_tokens, train_counts, ind, args.vocab_size, device)
            sums = data_batch.sum(1).unsqueeze(1)
            cnt += sums.sum(0).squeeze().cpu().numpy()
            if args.bow_norm:
                normalized_data_batch = data_batch / sums
            else:
                normalized_data_batch = data_batch
            theta, _ = model.get_theta(normalized_data_batch)
            thetaAvg += theta.sum(0).unsqueeze(0) / args.num_docs_train
            weighed_theta = sums * theta
            thetaWeightedAvg += weighed_theta.sum(0).unsqueeze(0)
            if idx % 100 == 0 and idx > 0:
                print('batch: {}/{}'.format(idx, len(indices)))
        thetaWeightedAvg = thetaWeightedAvg.squeeze().cpu().numpy() / cnt
        print('\nThe 10 most used topics are {}'.format(thetaWeightedAvg.argsort()[::-1][:10]))

        ## show topics
        beta = model.get_beta()
        topic_indices = list(np.random.choice(args.num_topics, 10)) # 10 random topics
        print('\n')
        for k in range(args.num_topics):#topic_indices:
            gamma = beta[k]
            top_words = list(gamma.cpu().numpy().argsort()[-args.num_words+1:][::-1])
            topic_words = [vocab[a] for a in top_words]
            print('Topic {}: {}'.format(k, topic_words))

        if args.train_embeddings:
            ## show etm embeddings 
            try:
                rho_etm = model.rho.weight.cpu()
            except:
                rho_etm = model.rho.cpu()
            queries = ['covid', 'refugee', 'resource', 'ventilator', 'women',
                        'men', 'money', 'society', 'health', 'people', 'family']
            
            print('\n')
            print('ETM embeddings...')
            for word in queries:
                print('word: {} .. etm neighbors: {}'.format(word, nearest_neighbors(word, rho_etm, vocab)))
            print('\n')




Visualizing model quality before training...
####################################################################################################
Visualize topics...
Topic 0: ['polycarbonate', 'fabricated', 'cardboard', 'fabricate', 'intransparent', 'cryptography', 'fablab', 'pvc', 'outline']
Topic 1: ['cytometry', 'opendata', 'serology', 'interoperable', 'disseminating', 'commoning', 'stamped', 'legislation', 'interconnection']
Topic 2: ['coronaviruses', 'silico', 'primers', 'virologists', 'cheapest', 'psychotherapists', 'thermal', 'inhibitors', 'drugstore']
Topic 3: ['dictionary', 'python', 'zip', 'dictionaries', 'closures', 'exposes', 'sliders', 'bibliography', 'attaches']
Topic 4: ['ppg', 'optimizer', '2017', 'glance', 'trophies', 'optimisation', 'concerts', 'opera', 'goals']
Topic 5: ['airlines', 'studios', 'virologists', 'agencies', 'pasteur', 'localities', 'tiers', 'silos', 'pulmonologists']
Topic 6: ['courage', 'python', 'sacrifices', 'friendships', 'bibliography', 'dependenc

/home/antoniorania/Desktop/EUvsVirus_ProjectClusters/src/ETM/utils.py:86: RuntimeWarning: invalid value encountered in true_divide
  ranks = ranks / denom


word: men .. neighbors: ['devpost', 'kaggle', 'gdpr', 'graphql', 'nltk', 'pitchvideo', 'flattenthecurve', 'prototipe', 'hackhaton', 'jitsi', 'tiktok', 'covid19', 'covid', 'glideapp', 'hackthon', 'digitaler', 'firestore', 'resnet', 'nanoencapsulation', 'desinfection']
vectors:  (10616, 300)
query:  (300,)
word: money .. neighbors: ['gdpr', 'jitsi', 'hackhaton', 'hackaton', 'flattenthecurve', 'covid', 'nltk', 'pitchvideo', 'balsamiq', 'airtable', 'digitaler', 'nanoencapsulation', 'graphql', 'euvsvirus', 'disinfodemic', 'pandemia', 'pandemy', 'desinfection', 'devpost', 'pseudonymisation']
vectors:  (10616, 300)
query:  (300,)
word: society .. neighbors: ['infodemic', 'airtable', 'pandemy', 'nltk', 'springboot', 'covid19', 'hackthon', 'gdpr', 'digitaler', 'pitchvideo', 'disinfodemic', 'pretrained', 'graphql', 'flattenthecurve', 'covid', 'pseudonymisation', 'kaggle', 'firestore', 'euvsvirus', 'pubchem']
vectors:  (10616, 300)
query:  (300,)
word: health .. neighbors: ['hackthon', 'hackhaton

Topic 102: ['underlined', 'underline', 'reflects', 'expressed', 'highlighted', 'instill', 'swiss', 'reinforces', 'ethical']
Topic 103: ['anticipate', 'oximeters', 'foresee', 'discern', 'lessen', 'sharpen', 'radiologists', 'perceive', 'modulate']
Topic 104: ['medtech', 'stethoscopes', 'biofilm', 'antibacterial', 'antimicrobial', 'calendars', 'advisors', 'clinician', 'pharma']
Topic 105: ['onboarded', 'stethoscopes', 'tracked', 'oximeters', 'hospitalizations', 'markers', 'axes', 'sanitizers', 'asynchronous']
Topic 106: ['alleviates', 'ethical', 'transdisciplinary', 'intercultural', 'align', 'disciplinary', 'embed', 'alleviate', 'underlined']
Topic 107: ['underlined', 'swiss', 'timezones', 'hydroxychloroquine', 'deepening', 'advertisers', 'underline', 'broadening', 'anosmia']
Topic 108: ['auscultation', 'touchless', 'stethoscopes', 'stethoscope', 'contactless', 'synchronizes', 'synchronise', 'ultrasonic', 'swiping']
Topic 109: ['auscultation', 'stethoscopes', 'ambu', 'stethoscope', 'under

****************************************************************************************************
Epoch----->11 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 699.28 .. NELBO: 699.28
****************************************************************************************************
****************************************************************************************************
VAL Doc Completion PPL: 10456.9
****************************************************************************************************
****************************************************************************************************
Epoch----->12 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 703.33 .. NELBO: 703.33
****************************************************************************************************
****************************************************************************************************
VAL Doc Completion PPL: 10448.8
*******************************************************************

...

****************************************************************************************************
Epoch----->71 .. LR: 0.005 .. KL_theta: 2.02 .. Rec_loss: 674.92 .. NELBO: 676.94
****************************************************************************************************
****************************************************************************************************
VAL Doc Completion PPL: 7266.2
****************************************************************************************************
****************************************************************************************************
Epoch----->72 .. LR: 0.005 .. KL_theta: 2.22 .. Rec_loss: 677.8 .. NELBO: 680.02
****************************************************************************************************
****************************************************************************************************
VAL Doc Completion PPL: 7234.9
********************************************************************

Topic 79: ['mentioned', 'explained', 'eradicated', 'utilised', 'clarified', 'elaborated', 'reactivated', 'additionally', 'mentioning']
Topic 80: ['studying', 'developped', 'reactivated', 'unfolding', 'integrated', 'synthesis', 'composing', 'recharging', 'multifunctional']
Topic 81: ['interdisciplinary', 'unrelated', 'disparate', 'pedestrian', 'merging', 'localized', 'obstructed', 'impacting', 'endangering']
Topic 82: ['narrowing', 'restricting', 'compromising', 'tweaking', 'delegating', 'limiting', 'excess', 'dividing', 'widening']
Topic 83: ['harms', 'inhibits', 'limb', 'damages', 'anosmia', 'protects', 'immunocompromised', 'mitigations', 'musculoskeletal']
Topic 84: ['organized', 'thankful', 'traveled', 'sane', 'hurdles', 'educated', 'survived', 'overcame', 'entertained']
Topic 85: ['referral', 'proprietary', 'ultrasound', 'technology', 'hybrid', 'antimicrobial', 'musculoskeletal', 'therapy', 'provider']
Topic 86: ['apprehend', 'assimilate', 'recognize', 'perceive', 'acquainted', 'co

****************************************************************************************************
Epoch----->81 .. LR: 0.005 .. KL_theta: 2.83 .. Rec_loss: 664.44 .. NELBO: 667.27
****************************************************************************************************
****************************************************************************************************
VAL Doc Completion PPL: 6967.7
****************************************************************************************************
****************************************************************************************************
Epoch----->82 .. LR: 0.005 .. KL_theta: 2.82 .. Rec_loss: 669.54 .. NELBO: 672.36
****************************************************************************************************
****************************************************************************************************
VAL Doc Completion PPL: 6945.2
*******************************************************************

Topic 78: ['incident', 'emergence', 'generation', 'twitter', 'ontology', 'century', 'tagging', 'wiki', 'django']
Topic 79: ['mentioned', 'explained', 'eradicated', 'utilised', 'mentioning', 'oder', 'clarified', 'elaborated', 'combating']
Topic 80: ['developped', 'studying', 'reactivated', 'integrated', 'unfolding', 'synthesis', 'composing', 'conversations', 'multifunctional']
Topic 81: ['interdisciplinary', 'unrelated', 'disparate', 'congested', 'merging', 'obstructed', 'impacting', 'endangering', 'localized']
Topic 82: ['restricting', 'tweaking', 'compromising', 'limiting', 'narrowing', 'excess', 'limit', 'widening', 'damaging']
Topic 83: ['harms', 'inhibits', 'limb', 'protects', 'damages', 'musculoskeletal', 'anosmia', 'impairment', 'mitigations']
Topic 84: ['organized', 'thankful', 'sane', 'traveled', 'educated', 'realized', 'entertained', 'occupied', 'overcame']
Topic 85: ['referral', 'technology', 'ultrasound', 'hybrid', 'proprietary', 'therapy', 'antimicrobial', 'musculoskeletal'

****************************************************************************************************
Epoch----->91 .. LR: 0.005 .. KL_theta: 3.53 .. Rec_loss: 659.41 .. NELBO: 662.94
****************************************************************************************************
****************************************************************************************************
VAL Doc Completion PPL: 6747.1
****************************************************************************************************
****************************************************************************************************
Epoch----->92 .. LR: 0.005 .. KL_theta: 3.64 .. Rec_loss: 663.39 .. NELBO: 667.03
****************************************************************************************************
****************************************************************************************************
VAL Doc Completion PPL: 6733.9
*******************************************************************

In [ ]:
model.eval()

Good resources for pre-trained models:

https://www.analyticsvidhya.com/blog/2019/03/pretrained-models-get-started-nlp/